# Projekt MSP1
Cílem tohoto projektu je se seznámit s programovými nástroji využívaných ve statistice a osvojit si základní procedury. Projekt není primárně zaměřen na efektivitu využívání programového vybavení (i když úplně nevhodné konstrukce mohou mít vliv na hodnocení), ale nejvíce nás zajímají vaše statistické závěry a způsob vyhodnocení. Dbejte také na to, že každý graf musí splňovat nějaké podmínky - přehlednost, čitelnost, popisky.

V projektu budete analyzovat časy běhu šesti různých konfigurací algoritmů. Ke každé konfiguraci vzniklo celkem 200 nezávislých běhů, jejichž logy máte k dispozici v souboru [logfiles.zip](logfiles.zip).

Pokud nemáte rozchozené prostředí pro pro spouštění Jupyter notebooku, můžete využití službu [Google Colab](https://colab.google/). Jakákoliv spolupráce, sdílení řešení a podobně je zakázána!

S případnými dotazy se obracejte na Vojtěcha Mrázka (mrazek@fit.vutbr.cz).

__Odevzdání:__ tento soubor (není potřeba aby obsahoval výstupy skriptů) do neděle 22. 10. 2023 v IS VUT. Kontrola bude probíhat na Pythonu 3.10.12; neočekává se však to, že byste používali nějaké speciality a nekompatibilní knihovny. V případě nesouladu verzí a podobných problémů budete mít možnost reklamace a prokázání správnosti funkce. Bez vyplnění vašich komentářů a závěrů do označených buněk nebude projekt hodnocen!

__Upozornění:__ nepřidávejte do notebooku další buňky, odpovídejte tam, kam se ptáme (textové komentáře do Markdown buněk)

__Tip:__ před odevzdáním resetujte celý notebook a zkuste jej spustit od začátku. Zamezíte tak chybám krokování a editací, kdy výsledek z buňky na konci použijete na začátku.

__OTÁZKA K DOPLNĚNÍ:__

_Jméno a login autora_

Jakub Kasem xkasem02

## Načtení potřebných knihoven
Načtěte knihovny, které jsou nutné pro zpracování souborů a práci se statistickými funkcemi. Není dovoleno načítat jiné knihovny.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from zipfile import ZipFile

## Načtení dat do DataFrame
Ze souboru `logfiles.zip` umístěném ve stejném adresáři načtěte data a vytvořte Pandas DataFrame.

Z logu vás budou nejvíce zajímat řádky
```
Configuration: config6
Run: 191
Time of run: 53.298725254089774
```

Můžete využít následující kostru - je vhodné pracovat přímo se ZIP souborem. Jedinou nevýhodou je to, že vám bude vracet _byte_ objekt, který musíte přes funkci `decode` zpracovat.

In [ ]:
def load_logfile(f) -> dict:
    """Load a logfile from a file-like object and return a dict with the data."""
    data = {
        "conf": None,
        "run": None,
        "time": np.nan
    }

    for line in f:
        line = line.decode("utf-8")
        if 'status:' in line.lower():
            continue
        if 'configuration:' in line.lower():
            data['conf'] = int(line.split(":")[1].strip().split('config')[1].strip())
            continue
        if 'time of run:' in line.lower():
            data['time'] = float(line.split(":")[1].strip())
            continue
        if 'run:' in line.lower():
            data['run'] = int(line.split(":")[1].strip())
            continue
    return data

data = []
with ZipFile("logfiles.zip") as zf:
    for filename in zf.namelist():
        with zf.open(filename, "r") as f:
            data.append(load_logfile(f))
df = pd.DataFrame(data)
df

## Analýza a čištění dat
Vhodným způsobem pro všechny konfigurace analyzujte časy běhů a pokud tam jsou, identifikujte hodnoty, které jsou chybné. 

In [ ]:
_df = df.copy()

sns.set(rc={'figure.figsize':(12, 8.5)})
fig = sns.boxplot(data=[_df.loc[_df['conf'] == confValue, 'time'].values for confValue in _df['conf'].unique()], 
                   orient='h', 
                   flierprops=dict(markerfacecolor='1'))
fig.set(xlabel="Time", ylabel="Conf", title='Graf pro identifikace outlieru')
plt.show()

__OTÁZKA K DOPLNĚNÍ:__

_Objevily se nějaké chybné hodnoty? Proč tam jsou s ohledem na to, že se jedná o běhy algoritmů?_

Áno, v grafe sú zobrazené odchýlky (biele kosoštvorce). Tie mohli byť napríklad okrajové prípady, alebo neobvyklé správanie počas behu konfigurácie.

Odchýlky môžu byť spôsobené aj chybami v procese zberu údajov, čo má vplyv na celkové rozdelenie. Okrem toho k prítomnosti odchýlok v súbore údajov môžu prispieť prirozdene náhodné prípady.

Vyčistěte dataframe `_df` tak, aby tam tyto hodnoty nebyly a ukažte znovu analýzu toho, že čištění dat bylo úspěšné. Odtud dále pracujte s vyčištěným datasetem.

In [ ]:
q1 = lambda x: x['time'].quantile(0.25)
q3 = lambda x:  x['time'].quantile(0.75)
iqr = lambda x: q3(x) - q1(x)
lowerBound = lambda x: q1(x) - 1.5 * iqr(x)
upperBound = lambda x: q3(x) + 1.5 * iqr(x)

df = df.groupby('conf', group_keys=False).apply(lambda x: x[(x['time'] >= lowerBound(x)) & (x['time'] <= upperBound(x))])

sns.set(rc={'figure.figsize':(10,8)})
fig = sns.boxplot(data=df, x='time', y='conf', orient='h')
fig.set(xlabel="Time", ylabel="Conf", title='Vycistena data')
plt.show()

## Deskriptivní popis hodnot
Vypište pro jednotlivé konfigurace základní deskriptivní parametry času pro jednotlivé konfigurace.  

__TIP__ pokud výsledky uložíte jako Pandas DataFrame, zobrazí se v tabulce.

In [ ]:
configurations = df.groupby('conf')['time']

descriptiveParameters = configurations.describe().drop(columns=['count'])
descriptiveParameters['meanGeometric'] = configurations.apply(stats.gmean)
descriptiveParameters['median'] = configurations.apply(np.median)
descriptiveParameters['variance'] = configurations.apply(np.var)
descriptiveParameters['skewness'] = configurations.apply(stats.skew)
descriptiveParameters['kurtosis'] = configurations.apply(stats.kurtosis)

descriptiveParameters = descriptiveParameters[['mean', 'meanGeometric', 'median', 'std', 'variance', 'min', '25%', '50%', '75%', 'max', 'skewness', 'kurtosis']]
descriptiveParameters

__OTÁZKA K DOPLNĚNÍ:__

_Okomentujte, co všechno můžeme z parametrů vyčíst._

- Stredná hodnota udáva priemerný čas behu pre každú konfiguráciu
- Medián a 50. percentil predstavujú strednú hodnotu, čo naznačuje, že polovica nameraných časov spadá pod túto hodnotu.
- Štandardná odchýlka meria rozptyl časov okolo priemeru. Vyššie štandardné odchýlky znamenajú väčšiu variabilitu.
- Rozptyl vyčísluje rozpätie údajov.
- Minimálna a maximálna hodnota predstavujú najnižší, resp. najvyšší zistený čas behu.
- Kvartily rozdeľujú údaje na štyri rovnaké časti.
- Šikmosť naznačuje asymetriu údajov. Kladné hodnoty naznačujú dlhší chvost vpravo, zatiaľ čo záporné hodnoty naznačujú dlhší chvost vľavo.

## Vizualizace
Vizualizujte časy běhů algoritmů v jednom kompaktním grafu tak, aby byl zřejmý i rozptyl hodnot. Zvolte vhodný graf, který pak níže komentujte.

In [ ]:
sns.set(rc={'figure.figsize':(10,8)})
fig = sns.boxplot(data=df, x='time', y='conf', orient='h')
fig.set(xlabel="Time", ylabel="Conf", title='Casy behu algoritmu')
plt.show()

__OTÁZKA K DOPLNĚNÍ:__

_Okomentujte  výsledky z tabulky._

- Najmenšiu minimálnu hodnotu má konfigurácia 4: $29.093401$ podľa deskriptívnej tabuľky.
- Najväčšiu minimálnu hodnotu má konfigurácia 5.
- Najmenší medián má konfigurácia 1: $99.799944$ podľa deskriptívnej tabuľky.
- Najväčší rozptyl má konfigurácia 4: $935.117531$ podľa deskriptívnej tabuľky.
- Naopak najmenší má konfigurácia 1: $78.206139$ podľa deskriptívnej tabuľky.

Z grafu som vyčítal, že kandidátmi na najrýchlejšiu konfiguráciu sú konfigurácie s číslami 1 a 4.

## Určení efektivity konfigurací algoritmů
Nás ale zajímá, jaká konfigurace je nejrychlejší. Z výše vykresleného grafu můžeme vyloučit některé konfigurace. Existuje tam však minimálně jedna dvojice, u které nedokážeme jednoznačně určit, která je lepší - pokud nebudeme porovnávat pouze extrémní hodnoty, které mohou být dané náhodou, ale celkově. Proto proveďte vhodný test významnosti - v následující části diskutujte zejména rozložení dat (i s odkazem na předchozí buňky, variabilitu vs polohu a podobně). Je nutné každý logický krok a výběry statistických funkcí komentovat. Můžete i přidat další buňky.

Vužijte vhodnou funkci z knihovny `scipy.stats` a funkci poté __implementujte sami__ na základě základních matematických funkcí knihovny `numpy` případně i funkcí pro výpočet studentova rozložení v [scipy.stats](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html). Při vlastní implementaci není nutné se primárně soustředit na efektivitu výpočtu (není potřeba využít všechny funkce numpy, můžete použít normální cykly a podobně - v hodnocení však bude zahrnuta přehlednost a neměly by se objevit jasné chyby, jako je zvýšení třídy složitosti a podobně).

__OTÁZKA K DOPLNĚNÍ:__

_Jaká data budete zkoumat? Jaké mají rozložení a parametry (např. varianci) a jaký test použijete? Jaká je nulová hypotéza? Jak se liší variabilita a poloha vybraných konfigurací?_

Budem skúmať časy behov algoritmov, konkrétne konfigurácie 1 a 4.

Keďže sa jedná o porovnávanie dvoch rôznych konfigurácií za predpokladu, že ide o normálne rozdelenie použijem **t test**. Na základe nerovností variancií sa bude jednať o Welchov t test.

Nulová hypotéza je: Medzi týmito dvoma konfiguráciami nie je významný rozdiel v čase behu.

Keď porovnáme šírky rámčekov v grafe (IQR) pre konfigurácie 1 a 4 zistíme, že konfigurácia 1 má nižšiu variabilitu ako konfigurácia 4. Konfigurácia s užším rámčekom (menšie IQR) má menšiu variabilitu v porovnaní s konfiguráciami s širšími rámčekmi.

In [ ]:
# Hodnoty priemeru konfigurácií 1 a 4.
# Potrebné pre určenie rýchlejšej konfigurácie, ak bude zamietnutá nulová hypotéza.
meanConf1 = descriptiveParameters['mean'][1]
meanConf4 = descriptiveParameters['mean'][4]

# Hodnoty rozptylu pre konfigurácie 1 a 4.
# Potrebné pre určenie parametra t testu.
varianceConf1 = descriptiveParameters['variance'][1]
varianceConf4 = descriptiveParameters['variance'][4]
equalVariances = varianceConf1 == varianceConf4

# Vykonanie dvojvýberového t testu medzi konfiguráciami 1 a 4,
# ktorý berie v potaz rozdielne rozptyly na základe parametra 'equal_var'.
statistic, p = stats.ttest_ind(df[df['conf'] == 1]['time'], df[df['conf'] == 4]['time'], equal_var=equalVariances)

print(f't-statistika: {statistic}')
print(f'rozdielny rozptyl: {not equalVariances}')
print(f'p-hodnota: {p}')
print(f'p < 0.05: {p < 0.05}')
print(f'mensia stredna hodnota: conf{1 if min(meanConf1, meanConf4) == meanConf1 else 2}')

__OTÁZKA K DOPLNĚNÍ:__

_Jaký je závěr statistického testu?_

Keďže p-hodnota testu je menšia ako $0.05$, časy behov týchto dvoch konfigurácií sa od seba významne **líšia**. To podporuje alternatívnu hypotézu, ktorá naznačuje, že jedna konfigurácia je rýchlejšia ako druhá.

Stredná hodnota času behov pre konfiguráciu 1 je menšia ako pre konfiguráciu 2, teda konfigurácia 1 má v priemere kratší čas behu na základe zozbieraných údajov.

In [ ]:
# Súbory času behov pre konfigurácie 1 a 4.
conf1Time = (df[df['conf'] == 1]['time']).to_numpy()
conf2Time = (df[df['conf'] == 4]['time']).to_numpy()

# Hodnoty priemeru konfigurácií 1 a 4.
# Potrebné pre určenie rýchlejšej konfigurácie, ak bude zamietnutá nulová hypotéza.
meanConf1 = descriptiveParameters['mean'][1]
meanConf4 = descriptiveParameters['mean'][4]

# Hodnoty rozptylu pre konfigurácie 1 a 4.
# Potrebné pre určenie parametra t testu.
varianceConf1 = descriptiveParameters['variance'][1]
varianceConf4 = descriptiveParameters['variance'][4]
equalVariances = varianceConf1 == varianceConf4

# Výpočet štandardnej odchýlky (výpočet bol vykonaný pri deskriptívnej tabuľke).
stdConf1 = descriptiveParameters['std'][1]
stdConf4 = descriptiveParameters['std'][4]

# Veľkosti vzoriek oboch konfigurácií.
n1 = len(conf1Time)
n2 = len(conf2Time)

# Výpočet stupňa voľnosti pre pomocou Welchovej aproximácie.
# df = ((std1**2/n1 + std2**2/n2)**2) / ((std1**2/n1)**2 / (n1 - 1) + (std2**2/n2)**2 / (n2 - 1))
degsOfFreedom = ((stdConf1**2 / n1 + stdConf4**2 / n2)**2) / ((stdConf1**2 / n1)**2 / (n1 - 1) + (stdConf4**2 / n2)**2 / (n2 - 1))

# Výpočet t-štatistiky pomocou vzorca pre dvojvýberový Welchov t-test.
# t = (mean1 - mean2) / sqrt((std1**2/n1) + (std2**2/n2))
tStatistic = (meanConf1 - meanConf4) / np.sqrt(stdConf1**2 / n1 + stdConf4**2 / n2)

# Výpočet dvojvýberovej p-hodnoty pomocou t-rozdelenia a stupňov voľnosti.
# p = 2 * (1 - cdf(abs(tsStatistic), df))
# cdf predstavuje hodnotu kumulatívnej distribučnej funkcie t-rozdelenia na základe absolútnej hodnoty t-štatistiky.
pValue = 2 * (1 - stats.t.cdf(np.abs(tStatistic), df=degsOfFreedom))

print(f't-statistika: {tStatistic}')
print(f'rozdielny rozptyl: {not equalVariances}')
print(f'p-hodnota: {pValue}')
print(f'p < 0.05: {pValue < 0.05}')
print(f'mensia stredna hodnota: conf{1 if min(meanConf1, meanConf1) == meanConf1 else 2}')
